<a href="https://colab.research.google.com/github/georgerieh/spotify-million-dataset/blob/main/Spotify_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cupy as cp
from cupyx.scipy.sparse import csr_matrix as cp_csr_matrix
import numpy as np
from tqdm import tqdm
import numpy as np
import torch
from scipy.sparse import csr_matrix as sp_csr_matrix

# Plan
## Part 1
Clean features, check variance, distribution and missing values
## Part 2
Compute correlations of continous features with followers, test categorical features with ANOVA/Kruskall-Wallis
## Part 3
Check redundancy between features

## Part 4
Fit a simple model to gauge feature importance




## Part 0
Load the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !unzip '/content/drive/MyDrive/spotify_million_playlist_dataset.zip' -d 'spotify_million_playlist_dataset'

In [ ]:
import pandas as pd
import json
import glob
from tqdm import tqdm
def load_playlists(path_pattern):
    all_data = []
    for file_name in tqdm(glob.glob(path_pattern), desc='Files'):
        with open(file_name, 'r') as f:
            data = json.load(f)
            playlists = data['playlists']
            for playlist in playlists:
                for song in playlist['tracks']:
                    yield {'name': song['track_name'], 'artist': song['artist_name'], 'dur': song['duration_ms'], 'album': song['album_name'], 'p_name': playlist['name'], 'is_shared_p': bool(playlist['collaborative']), 'n_tracks': playlist['num_tracks'], 'n_albums': playlist['num_albums'], 'n_followers': playlist['num_followers']}



In [ ]:
import json
import glob
import csv
import hashlib

def stable_hash(s):
    return int(hashlib.md5(s.encode('utf-8')).hexdigest(), 16)

with open("interactions.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["playlist_id", "song_id", "weight"])
    for file_name in tqdm(glob.glob("spotify_million_playlist_dataset/data/*.json")):
        with open(file_name, "r") as jf:
            data = json.load(jf)
            for playlist in data["playlists"]:
                pid = stable_hash(str(playlist["pid"]))
                for track in playlist["tracks"]:
                    sid = stable_hash(track["track_uri"])
                    writer.writerow([pid, sid, 1])


100%|██████████| 1000/1000 [11:04<00:00,  1.51it/s]


In [ ]:
import pandas as pd
from collections import Counter

song_counts = Counter()
playlist_counts = Counter()

for chunk in tqdm(pd.read_csv(
    "interactions.csv",
    chunksize=1_000_000,   # tune this
)):
    song_counts.update(chunk["song_id"])
    playlist_counts.update(chunk["playlist_id"])


28it [00:59,  1.95s/it]

In [ ]:
playlist_map = {}
song_map = {}

next_playlist_idx = 0
next_song_idx = 0

for chunk in tqdm(pd.read_csv("interactions.csv", chunksize=1_000_000)):
    for row in chunk.itertuples():
        pid_hash = int(row.playlist_id)
        sid_hash = int(row.song_id)
        if pid_hash not in playlist_map:
            playlist_map[int(pid_hash)] = next_playlist_idx
            next_playlist_idx += 1
        if sid_hash not in song_map:
            song_map[int(sid_hash)] = next_song_idx
            next_song_idx += 1

with open('playlist_map.json', 'w') as pl, open('song_map.json', 'w') as so,
  json.dump(playlist_map, pl)
  json.dump(song_map, so)

67it [03:34,  3.20s/it]


In [ ]:
from scipy.sparse import coo_matrix, save_npz, load_npz

rows = []
cols = []
data_vals = []

for chunk in tqdm(pd.read_csv("interactions.csv", chunksize=1_000_000)):
    for row in chunk.itertuples():
        pid_idx = playlist_map[int(row.playlist_id)]
        sid_idx = song_map[int(row.song_id)]

        rows.append(pid_idx)
        cols.append(sid_idx)
        data_vals.append(1)  # binary interaction

interaction_matrix = coo_matrix(
    (data_vals, (rows, cols)),
    shape=(len(playlist_map), len(song_map))
)

save_npz('interaction.npz', interaction_matrix)
del interaction_matrix


67it [03:44,  3.35s/it]


In [ ]:
interaction_matrix = load_npz('interaction_npz')
interaction_matrix = interaction_matrix.tocsr()
save_npz('interaction_csr.npz', interaction_matrix)
del interaction_matrix


In [ ]:
all_recommendations = {}  # key = playlist index, value = list of top-n songs

def save_recommendations(playlist_idx, recommended_songs):
    all_recommendations[playlist_idx] = recommended_songs


In [ ]:

def csr_to_torch_sparse(csr_matrix):
    coo = csr_matrix.tocoo()  # must be COO
    if coo.data.size == 0:
        # empty matrix → return a 0-valued sparse tensor
        indices = torch.zeros((2,0), dtype=torch.long)
        values  = torch.zeros((0,), dtype=torch.float32)
    else:
        indices = torch.tensor([coo.row, coo.col], dtype=torch.long)
        values  = torch.tensor(coo.data, dtype=torch.float32)  # must be float
    shape = coo.shape
    return torch.sparse_coo_tensor(indices, values, size=shape)

def compute_topk_similarity_gpu(
    sparse_x_batch_cpu: sp_csr_matrix,
    csr_matrix_all_cupy: cp_csr_matrix,
    top_k: int = 5,
    zero_self_similarity: bool = False,
    batch_offset: int = 0,
):
    # Convert CPU batch to GPU (CuPy)
    sparse_x_batch_gpu = cp_csr_matrix(sparse_x_batch_cpu)

    # Calculate L2 norms for normalization
    # Example: If a row is [3, 0, 4], power(2) is [9, 0, 16], sum is 25, sqrt is 5.
    all_norms = cp.sqrt(csr_matrix_all_cupy.power(2).sum(axis=1)).ravel()
    batch_norms = cp.sqrt(sparse_x_batch_gpu.power(2).sum(axis=1)).ravel()

    # Compute sparse dot product (Batch Matrix Multiplication)
    # ====== #
    # sparse_x_batch (2 rows) = [[1, 0], [0, 1]]
    # csr_matrix_all (3 rows) = [[1, 1], [1, 0], [0, 1]]
    # sim = batch @ all.T
    # sim result shape: (2, 3) -> Matrix of dot products
    # sim = [[1, 1, 0],
    #        [1, 0, 1]]
    # ====== #
    sim = sparse_x_batch_gpu @ csr_matrix_all_cupy.T

    if cp.sparse.issparse(sim):
        sim = sim.toarray().astype('f')

    # Normalize similarity scores
    # ====== #
    # batch_norms = [1.0, 1.0] (shape 2,)
    # all_norms = [1.41, 1.0, 1.0] (shape 3,)
    # batch_norms[:, None] transforms to (2, 1) column
    # all_norms[None, :] transforms to (1, 3) row
    # Multiplying them creates a (2, 3) denominator matrix:
    # [[1.41, 1.0, 1.0],
    #  [1.41, 1.0, 1.0]]
    # sim = sim / (denominator + 1e-8)
    # ====== #
    sim = sim / (batch_norms[:, None] * all_norms[None, :] + 1e-8)

    # Optional: remove self-similarity
    if zero_self_similarity:
        rows_in_batch = sparse_x_batch_gpu.shape[0]
        for i in tqdm(range(rows_in_batch)):
            global_idx = batch_offset + i
            if global_idx < csr_matrix_all_cupy.shape[0]:
                # Sets the diagonal element (the item vs itself) to 0
                sim[i, global_idx] = 0.0

    # Top-k selection (GPU)
    # ====== #
    # sim = [[0.7, 1.0, 0.2],
    #        [0.1, 0.5, 0.9]]
    # cp.argsort(sim, axis=1) ->
     #[[2, 0, 1],
     #[0, 1, 2]] (indices of values low to high)
    # [:, ::-1] reverses it    -> [[1, 0, 2], [2, 1, 0]] (indices of values high to low)
    # [:, :top_k] (top_k=2)    -> [[1, 0], [2, 1]]
    # ====== #
    topk_idx = cp.argsort(sim, axis=1)[:, ::-1][:, :top_k]

    # Use take_along_axis to get the corresponding values
    # ====== #
    # From sim, pick values at indices [[1, 0], [2, 1]]
    # topk_vals = [[1.0, 0.7], [0.9, 0.5]]
    # ====== #
    topk_vals = cp.take_along_axis(sim, topk_idx, axis=1)

    return topk_idx.get(), topk_vals.get()

In [ ]:
import numpy as np
import cupy as cp
from cupy.sparse import csr_matrix as cp_csr_matrix
from scipy.sparse import csr_matrix as sp_csr_matrix

# Convert the interaction_matrix to CuPy's sparse matrix format once
# This will be used for GPU computations.
# The original scipy.sparse.csr_matrix is still named 'interaction_matrix'
# and will be used by functions like gather_candidate_songs and score_candidate_songs
# that are not yet cupy-enabled.
interaction_matrix_gpu = cp_csr_matrix(interaction_matrix.astype('f'))
def score_candidate_songs(candidate_songs_indices, top_similar_playlist_indices, top_similar_scores, interaction_matrix):
    scores = {}
    for song_idx in candidate_songs_indices:
        score = 0.0
        for i, sim_playlist_idx in enumerate(top_similar_playlist_indices):
            # Check if the song is in the similar playlist
            if interaction_matrix[sim_playlist_idx, song_idx] > 0:
                score += top_similar_scores[i]  # Add the similarity score
        scores[song_idx] = score
    return scores
# Redefine the compute_topk_similarity_gpu function to use CuPy

import cupy as cp
from cupyx.scipy.sparse import csr_matrix as cp_csr_matrix
import numpy as np

# 1. SETUP
interaction_matrix = load_npz('interaction_csr.npz').astype('f')
interaction_matrix_gpu = cp_csr_matrix(interaction_matrix)

batch_size = 512
top_k = 5
top_n = 500  # Based on your goal of 500 songs

for batch_start in tqdm(range(0, interaction_matrix_gpu.shape[0], batch_size)):
    batch_end = min(batch_start + batch_size, interaction_matrix_gpu.shape[0])
    current_batch_size = batch_end - batch_start

    target_batch_gpu = interaction_matrix_gpu[batch_start:batch_end, :]

    # STEP A: GPU Similarity Search (Assuming this function returns CuPy arrays)
    topk_indices, topk_scores = compute_topk_similarity_gpu(
        target_batch_gpu,
        interaction_matrix_gpu,
        top_k=top_k,
        batch_offset=batch_start
    )

    # STEP B: Vectorized Scoring via Dot Product
    # We create a "Weight Matrix" of shape (batch_size, top_k * batch_size)
    # but it's easier to just use a sparse construction.

    # Flatten neighbor indices and scores
    flat_neighbors = cp.array(topk_indices.ravel())
    neighbor_matrix = interaction_matrix_gpu[flat_neighbors] # Shape: (batch_size * top_k, num_songs)

    # Create a sparse weight matrix to perform the "Sum"
    # rows: [0,0,0,0,0, 1,1,1,1,1, ...]
    # cols: [0,1,2,3,4, 5,6,7,8,9, ...]
    row_idx = cp.repeat(cp.arange(current_batch_size), top_k)
    col_idx = cp.arange(current_batch_size * top_k)
    weights_sparse = cp_csr_matrix((cp.array(topk_scores.ravel()), (row_idx, col_idx)),
                                   shape=(current_batch_size, current_batch_size * top_k))

    # The Resulting batch_scores is (batch_size, num_songs)
    batch_scores = weights_sparse.dot(neighbor_matrix)

    # STEP C: Masking (Exclude songs user already has)
    # We convert batch_scores to dense only for the Top-N selection to save memory
    batch_scores = batch_scores.toarray()

    # Masking: set already-seen songs to a very low value
    # target_batch_gpu.nonzero() returns (rows, cols) of existing tracks
    nz_rows, nz_cols = target_batch_gpu.nonzero()
    batch_scores[nz_rows, nz_cols] = -1e8

    # STEP D: Pick Top N (Vectorized)
    # Using argpartion is O(n), much faster than sorting the whole array
    partitioned_indices = cp.argpartition(-batch_scores, top_n, axis=1)[:, :top_n]

    # Sort only the top_n results
    row_selector = cp.arange(current_batch_size)[:, None]
    top_scores = batch_scores[row_selector, partitioned_indices]
    final_sort = cp.argsort(-top_scores, axis=1)
    final_indices = cp.take_along_axis(partitioned_indices, final_sort, axis=1)

    # STEP E: Save results
    final_recs_cpu = final_indices.get()
    for i in range(current_batch_size):
        save_recommendations(batch_start + i, final_recs_cpu[i].tolist())

    # Clean up
    cp.get_default_memory_pool().free_all_blocks()

100%|██████████| 100/100 [00:00<00:00, 63550.06it/s]

100%|██████████| 100/100 [00:00<00:00, 76052.66it/s]

100%|██████████| 100/100 [00:00<00:00, 98273.29it/s]

100%|██████████| 100/100 [00:00<00:00, 97360.82it/s]

100%|██████████| 100/100 [00:00<00:00, 53946.03it/s]

100%|██████████| 100/100 [00:00<00:00, 65048.14it/s]

100%|██████████| 100/100 [00:00<00:00, 86338.08it/s]

100%|██████████| 100/100 [00:00<00:00, 31758.19it/s]

100%|██████████| 100/100 [00:00<00:00, 91678.78it/s]

100%|██████████| 100/100 [00:00<00:00, 88975.48it/s]

100%|██████████| 100/100 [00:00<00:00, 95390.13it/s]

100%|██████████| 100/100 [00:00<00:00, 30266.30it/s]

100%|██████████| 100/100 [00:00<00:00, 93748.41it/s]

100%|██████████| 100/100 [00:00<00:00, 86766.74it/s]

100%|██████████| 100/100 [00:00<00:00, 89051.04it/s]

100%|██████████| 100/100 [00:00<00:00, 140371.62it/s]

  0%|          | 15/10000 [05:59<66:25:57, 23.95s/it]


KeyboardInterrupt: 

In [ ]:
interaction_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 65464776 stored elements and shape (1000000, 2262292)>

In [ ]:


from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
print(torch.cuda.memory_reserved())  # total reserved by PyTorch
print(torch.cuda.memory_allocated())

13895729152
4984684544


# Task
Accelerate the playlist recommendation system by configuring the Colab environment for GPU usage, converting the `interaction_matrix` to a `cupy.sparse.csr_matrix`, and implementing a GPU-accelerated version of the `compute_topk_similarity` function to improve performance.

## Configure GPU Environment

### Subtask:
Configure the Colab runtime for GPU usage and import the CuPy library.


```markdown
Before proceeding, please ensure that your Colab runtime is set to GPU. To do this, go to `Runtime` > `Change runtime type`, select `T4 GPU` (or the best available GPU), and click `Save`.

Once the runtime is configured, execute the next cell to install and import `CuPy` and verify GPU availability.
```

**Reasoning**:
Now that the user has been reminded to set the GPU runtime, the next step is to install and import the `cupy` library and then verify its successful setup by checking for GPU availability, as per the subtask instructions.



In [ ]:
!pip install cupy-cuda11x # Install CuPy for CUDA 11.x, adjust if using a different CUDA version
import cupy as cp

print("CuPy imported successfully.")
if cp.cuda.is_available():
    print("GPU is available and detected by CuPy.")
    print(f"CuPy backend: {cp.cuda.runtime.getDeviceCount()} devices available.")
else:
    print("GPU is not available or not detected by CuPy.")